In [1]:
import os
import torch
import pickle
import numpy as np
import matplotlib.pyplot as plt  


from auto_encoder import EncoderLSTM, DecoderLSTM
from data_handler import load_sm_data
from trainer import train_AutoEncoder
from utils import restore_AE_checkpoint


2024-04-24 07:12:29.408778: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
class Option:
    module = "LSTM"
    num_layer = 2 #3
    hidden_dim = int(32**2/2) #32
    batch_size = 32
    norm = False
    alpha_norm = 1e-4
    z_dim = 1
    seq_len = 96
    data_name = "sm_consumption"
    lr = 0.001
    beta1 = 0.9
    epoch = 200
    size_data = 1974
    n_epochs = 500
    
    def __init__(self):
        pass

opt = Option()


In [3]:
ori_data = load_sm_data(opt)
dataset = np.stack(ori_data).reshape(-1, 96)

sm_consumption dataset is ready.


In [4]:
# Load scaler_params  
with open("scaler_params.pkl", "rb") as f:  
    scaler_params = pickle.load(f)  

    
# Let's say you want to invert the scaling for the first row  
row = 0  
data_min, data_max = scaler_params[row]  
scaled_values = dataset[row, :]  
original_values = scaled_values * (data_max - data_min) + data_min  


In [ ]:
ER_dir = f"./AE_trained/checkpoints/{opt.module}/{opt.num_layer}-l/{opt.hidden_dim}-Hdim/{opt.batch_size}-bs{['/n-norm',f'/y-norm/{opt.alpha_norm}-an'][opt.norm]}"
ckpt = sorted([ int(i[11:-4]) for i in os.listdir(ER_dir)])[-1]
ckpt_dir = os.path.join(ER_dir, f"checkpoint_{ckpt}.pth")
print(ckpt_dir)


device = "cpu"
nete = EncoderLSTM(input_size=opt.z_dim, 
                       hidden_dim=opt.hidden_dim, 
                       batch_size=opt.batch_size, 
                       n_layers=opt.num_layer,
                       device=device).to(device)
    
netr = DecoderLSTM(hidden_dim=opt.hidden_dim,
                   output_size=opt.z_dim,
                   batch_size=opt.batch_size,
                   n_layers=opt.num_layer,
                   forecasting_horizon=opt.seq_len,
                   device=device).to(device)

state_ER = {'encoder': nete, 'decoder': netr}
state_ER = restore_AE_checkpoint(ckpt_dir, state_ER, device)

input_data = torch.tensor(ori_data, dtype=torch.float32).to(device)


encoder = state_ER['encoder']
encoder = encoder.to(device)

decoder = state_ER['decoder']
decoder = decoder.to(device)

with torch.no_grad():
    
    encoder_hidden = encoder.init_hidden(batch_size=input_data.shape[0])
    output_encoder, hidden_embeddings = encoder.forward(input_data, encoder_hidden)
    input_decoder = hidden_embeddings[0][-1, :, :].unsqueeze(0).permute(1, 0, 2)
    reconstructed_data, decoder_hidden, _ = decoder.forward(decoder_input=input_decoder, encoder_hidden=hidden_embeddings)
    

./AE_trained/checkpoints/LSTM/2-l/512-Hdim/32-bs/n-norm/checkpoint_30.pth


/var/tmp/ipykernel_13417/2124805109.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  input_data = torch.tensor(ori_data, dtype=torch.float32).to(device)


In [ ]:
def visualize_time_series(time_series_tensor,reconstructed, i):  
    plt.figure(figsize=(14,6))  
    plt.plot(time_series_tensor[i,:,0])  
    plt.plot(reconstructed[i,:,0], c="r")  
    plt.title("Time Series Visualization")  
    plt.xlabel("Time steps")  
    plt.ylabel("Value")  
    plt.show()  

i = 2
# To use it  
visualize_time_series(input_data, reconstructed_data, i) 
visualize_time_series(input_data, reconstructed_data, 200) 


In [ ]:
normalized_embs = []
for hidden_emb in hidden_embeddings:
    # Normalize the tensor  
    mean = hidden_emb.mean()  
    std = hidden_emb.std()  
    normalized = (hidden_emb - mean) / std  
    normalized_embs.append(normalized)


# Concatenate the tensors along the last dimension  
combined = torch.cat(normalized_embs, dim=2) # Shape: [2, 1974, 64]  

# Function to visualize a tensor as an image  
def visualize_tensor_as_image(tensor, index, i):  
    image_data = tensor[i, index, :].reshape(1, 32, 32).permute(1, 2, 0)  
    plt.imshow(image_data)  
    plt.show()  
  
# Visualize the first image  
for dim_ in range(2):
    visualize_tensor_as_image(combined, 2, dim_)
    visualize_tensor_as_image(combined, 200, dim_)


In [ ]:
from ipywidgets import interact  
  
def visualize(i):  
    #visualize_tensor_as_image(combined, i)
    visualize_time_series(input_data, reconstructed_data, i)  
    
  
interact(visualize, i=(0, input_data.shape[0]-1))  
